# Setup clickhouse client

In [1]:
pip install clickhouse-connect

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [108]:
import clickhouse_connect
import glob
import numpy as np
import psutil
import matplotlib.pyplot as plt
import pandas as pd
import time
from collections import defaultdict

In [4]:
ch_host='localhost'
ch_port=8123

In [5]:
client = clickhouse_connect.get_client(host=ch_host, port=ch_port)

# Execute Queries

In [58]:
activate_optimizer_setting = " SETTINGS yannakakis_optimizer = 1, max_execution_time=50"
deactivate_optimizer_setting = " SETTINGS yannakakis_optimizer = 0, max_execution_time=50"
test_query = "SELECT * FROM system.tables"

In [59]:
# Execute Test Query with and without optimization
client.query((test_query + activate_optimizer_setting)).result_rows
client.query((test_query + deactivate_optimizer_setting)).result_rows

[('INFORMATION_SCHEMA',
  'COLUMNS',
  UUID('00000000-0000-0000-0000-000000000000'),
  'View',
  0,
  [],
  '',
  datetime.datetime(1970, 1, 1, 2, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'CEST')),
  [],
  [],
  "CREATE VIEW INFORMATION_SCHEMA.COLUMNS (`table_catalog` String, `table_schema` String, `table_name` String, `column_name` String, `ordinal_position` UInt64, `column_default` String, `is_nullable` String, `data_type` String, `character_maximum_length` Nullable(UInt64), `character_octet_length` Nullable(UInt64), `numeric_precision` Nullable(UInt64), `numeric_precision_radix` Nullable(UInt64), `numeric_scale` Nullable(UInt64), `datetime_precision` Nullable(UInt64), `character_set_catalog` Nullable(String), `character_set_schema` Nullable(String), `character_set_name` Nullable(String), `collation_catalog` Nullable(String), `collation_schema` Nullable(String), `collation_name` Nullable(String), `domain_catalog` Nullable(String), `domain_schema` Nullable(String)

In [60]:
path1 = 'queries/imdb/'
path2 = 'queries/snap/'
paths = [path1, path2]

# Use glob to find all files with a .sql extension in the specified directory
sql_files = []
for path in paths:
    sql_files.extend(glob.glob(path + '*.sql'))
queries = []

# Loop through each SQL file and read its contents
for sql_file in sql_files:
    with open(sql_file, 'r') as file:
        queries.append([file.read(), sql_file])

In [61]:
# key = filename
# value = [elapsed_time, memory, query_result, optimizer_used, run_number]
results = defaultdict(list)

excluded = []
runs = 3

In [62]:
def run_queries(number_of_runs, queries, query_settings):
    for i in range(number_of_runs):
        for query, filename in queries:
            if any(excluded_file in filename for excluded_file in excluded):
                continue
            print(i, filename)
            start_time = time.time()
            initial_memory = psutil.virtual_memory().used

            try:
                result = client.query(query + query_settings)
            except Exception as e:
                print(f"Query execution failed for {filename}: {e}")
                results[filename].append([-1, -1, "TIMEOUT_EXCEEDED", query_settings, i])
                continue
                
            end_time = time.time()
            elapsed_time = end_time - start_time
            final_memory = psutil.virtual_memory().used
            peak_memory = max(initial_memory, final_memory) >> 20 # convert to MB
            results[filename].append([elapsed_time, peak_memory, result.result_rows, query_settings, i])

## Run with optimizer

In [63]:
run_queries(runs, queries, activate_optimizer_setting)

0 queries/imdb/q2d.sql
0 queries/imdb/20a.sql
0 queries/imdb/q2c.sql
0 queries/imdb/3b.sql
0 queries/imdb/q5b.sql
0 queries/imdb/17d.sql
0 queries/imdb/3a.sql
0 queries/imdb/q5a.sql
0 queries/imdb/17e.sql
0 queries/imdb/q2a.sql
0 queries/imdb/17a.sql
0 queries/imdb/20b.sql
0 queries/imdb/q2b.sql
0 queries/imdb/17c.sql
0 queries/imdb/3c.sql
0 queries/imdb/17b.sql
0 queries/imdb/q5c.sql
0 queries/snap/patents-path04.sql
0 queries/snap/patents-path05.sql
0 queries/snap/patents-path02.sql
0 queries/snap/patents-path03.sql
1 queries/imdb/q2d.sql
1 queries/imdb/20a.sql
1 queries/imdb/q2c.sql
1 queries/imdb/3b.sql
1 queries/imdb/q5b.sql
1 queries/imdb/17d.sql
1 queries/imdb/3a.sql
1 queries/imdb/q5a.sql
1 queries/imdb/17e.sql
1 queries/imdb/q2a.sql
1 queries/imdb/17a.sql
1 queries/imdb/20b.sql
1 queries/imdb/q2b.sql
1 queries/imdb/17c.sql
1 queries/imdb/3c.sql
1 queries/imdb/17b.sql
1 queries/imdb/q5c.sql
1 queries/snap/patents-path04.sql
1 queries/snap/patents-path05.sql
1 queries/snap/paten

## Run without optimizer

In [64]:
run_queries(runs, queries, deactivate_optimizer_setting)

0 queries/imdb/q2d.sql
0 queries/imdb/20a.sql
0 queries/imdb/q2c.sql
0 queries/imdb/3b.sql
0 queries/imdb/q5b.sql
0 queries/imdb/17d.sql
Query execution failed for queries/imdb/17d.sql: :HTTPDriver for http://localhost:8123 returned response code 408)
 Code: 159. DB::Exception: Timeout exceeded: elapsed 50.005893476 seconds, maximum: 50. (TIMEOUT_EXCEEDED) (version 23.9.1.1)

0 queries/imdb/3a.sql
0 queries/imdb/q5a.sql
0 queries/imdb/17e.sql
Query execution failed for queries/imdb/17e.sql: :HTTPDriver for http://localhost:8123 returned response code 408)
 Code: 159. DB::Exception: Timeout exceeded: elapsed 50.010444622 seconds, maximum: 50. (TIMEOUT_EXCEEDED) (version 23.9.1.1)

0 queries/imdb/q2a.sql
0 queries/imdb/17a.sql
Query execution failed for queries/imdb/17a.sql: :HTTPDriver for http://localhost:8123 returned response code 408)
 Code: 159. DB::Exception: Timeout exceeded: elapsed 50.004581214 seconds, maximum: 50. (TIMEOUT_EXCEEDED) (version 23.9.1.1)

0 queries/imdb/20b.sql


In [95]:
benchmark_results = {}
for filename, result_objects in results.items():
    _, dataset, query_name = filename.split("/")
    for result_object in result_objects: #result_object = [elapsed_time, memory, query_result, optimizer_used, run_number]
        elapsed_time, memory, query_result, optimizer_used, run_number = result_object
        if activate_optimizer_setting in optimizer_used:
            optimizer = True
        else:
            optimizer = False
        benchmark_results[(dataset,query_name,run_number,optimizer)] = result_object
print(benchmark_results)

{('imdb', 'q2d.sql', 0, True): [2.3228983879089355, 3533529088, [('& Teller',)], ' SETTINGS yannakakis_optimizer = 1, max_execution_time=50', 0], ('imdb', 'q2d.sql', 1, True): [3.577461004257202, 3433906176, [('& Teller',)], ' SETTINGS yannakakis_optimizer = 1, max_execution_time=50', 1], ('imdb', 'q2d.sql', 2, True): [3.750791549682617, 3575820288, [('& Teller',)], ' SETTINGS yannakakis_optimizer = 1, max_execution_time=50', 2], ('imdb', 'q2d.sql', 0, False): [4.224424839019775, 4272914432, [('& Teller',)], ' SETTINGS yannakakis_optimizer = 0, max_execution_time=50', 0], ('imdb', 'q2d.sql', 1, False): [4.457141399383545, 6280777728, [('& Teller',)], ' SETTINGS yannakakis_optimizer = 0, max_execution_time=50', 1], ('imdb', 'q2d.sql', 2, False): [4.6254658699035645, 6114254848, [('& Teller',)], ' SETTINGS yannakakis_optimizer = 0, max_execution_time=50', 2], ('imdb', '20a.sql', 0, True): [13.011408567428589, 4068147200, [('',)], ' SETTINGS yannakakis_optimizer = 1, max_execution_time=50

# Tables

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [122]:
df = pd.DataFrame([[k[0],k[1],k[2],k[3],v[0],v[1]] for k,v in benchmark_results.items()]) \
           .rename(columns={0:'Dataset',1:'Query',2:'Run Number',3:'Optimizer Used', 4:'Runtime (s)', 5:'Memory Usage (MB)'})
df

,Dataset,Query,Run Number,Optimizer Used,Runtime (s),Memory Usage (MB)
0,imdb,q2d.sql,0,True,2.322898,3533529088
1,imdb,q2d.sql,1,True,3.577461,3433906176
2,imdb,q2d.sql,2,True,3.750792,3575820288
3,imdb,q2d.sql,0,False,4.224425,4272914432
4,imdb,q2d.sql,1,False,4.457141,6280777728
5,imdb,q2d.sql,2,False,4.625466,6114254848
6,imdb,20a.sql,0,True,13.011409,4068147200
7,imdb,20a.sql,1,True,20.384899,4115181568
8,imdb,20a.sql,2,True,20.927603,4193726464
9,imdb,20a.sql,0,False,18.439534,6647570432


## Runtime Table Overview

In [124]:
# Calculate average runtime with and without optimizer
summary_df = df.groupby(['Dataset', 'Query']).apply(lambda x: pd.Series({
    'Without Optimizer': x[x['Optimizer Used'] == False]['Runtime (s)'].mean(),
    'With Optimizer': x[x['Optimizer Used'] == True]['Runtime (s)'].mean()
})).reset_index()

# Compute absolute difference between runtime with and without optimizer
summary_df['Absolute Difference Runtime'] = summary_df['With Optimizer']-summary_df['Without Optimizer']

# Print the summary DataFrame
print("Summary for Runtime:")
summary_df

Summary DataFrame for Runtime:


,Dataset,Query,Without Optimizer,With Optimizer,Absolute Difference Runtime
0,imdb,17a.sql,-1.000000,14.823390,15.823390
1,imdb,17b.sql,-1.000000,14.930989,15.930989
2,imdb,17c.sql,-1.000000,14.328009,15.328009
3,imdb,17d.sql,-1.000000,14.262763,15.262763
4,imdb,17e.sql,-1.000000,15.443735,16.443735
5,imdb,20a.sql,20.302492,18.107970,-2.194522
6,imdb,20b.sql,17.867013,15.985952,-1.881062
7,imdb,3a.sql,4.634575,3.921755,-0.712820
8,imdb,3b.sql,3.007545,2.540017,-0.467529
9,imdb,3c.sql,6.011509,3.619442,-2.392066


## Memory Usage Overview

In [127]:
memory_df = df.groupby(['Dataset', 'Query']).apply(lambda x: pd.Series({
    'Without Optimizer': x[x['Optimizer Used'] == False]['Memory Usage (MB)'].mean(),
    'With Optimizer': x[x['Optimizer Used'] == True]['Memory Usage (MB)'].mean()
})).reset_index()

memory_df['Absolute Difference Memory (MB)'] = memory_df['With Optimizer']-memory_df['Without Optimizer']


print("Summary for Memory:")
memory_df

Summary for Memory:


,Dataset,Query,Without Optimizer,With Optimizer,Absolute Difference Memory (MB)
0,imdb,17a.sql,-1.000000e+00,3.834100e+09,3.834100e+09
1,imdb,17b.sql,-1.000000e+00,3.940131e+09,3.940131e+09
2,imdb,17c.sql,-1.000000e+00,3.760401e+09,3.760401e+09
3,imdb,17d.sql,-1.000000e+00,3.853225e+09,3.853225e+09
4,imdb,17e.sql,-1.000000e+00,3.657508e+09,3.657508e+09
5,imdb,20a.sql,6.672287e+09,4.125685e+09,-2.546602e+09
6,imdb,20b.sql,5.712344e+09,3.834444e+09,-1.877900e+09
7,imdb,3a.sql,5.183971e+09,3.646737e+09,-1.537234e+09
8,imdb,3b.sql,7.126435e+09,4.049697e+09,-3.076738e+09
9,imdb,3c.sql,4.922383e+09,3.761463e+09,-1.160920e+09
